# PaymanAI

PaymanAI 提供代表 AI Agent 进行支付（法币和加密货币）的功能。开始之前：

1. 在 app.paymanai.com **注册**以创建 AI Agent 并获取您的 **API Key**。
2. **设置**环境变量（`PAYMAN_API_SECRET` 用于您的 API Key，`PAYMAN_ENVIRONMENT` 用于沙盒或生产环境）。

本 Notebook 提供了将 PaymanAI 集成到 LangChain 作为工具的快速概述。如需完整参考，请参阅 API 文档。

## 概述

PaymanAI 集成是 `langchain-community`（或您自定义的）包的一部分。它允许您：

- 向加密货币地址或银行账户**发送**支付 (`send_payment`)。
- **搜索**收款人 (`search_payees`)。
- **添加**新的收款人 (`add_payee`)。
- 使用托管的结账链接向客户**收款** (`ask_for_money`)。
- **检查** Agent 或客户的余额 (`get_balance`)。

这些可以被包装成 **LangChain Tools**，供基于 LLM 的 Agent 自动调用。

### 集成详情

| Class | Package | Serializable | JS support | Package latest |
| :--- | :--- | :---: | :---: | :--- |
| PaymanAI | `langchain_community` | ❌ | ❌ | [PyPI Version] |

如果您只是调用 PaymanAI SDK，可以直接调用，也可以通过 LangChain 中的 **Tool** 接口进行调用。

## 设置

1. **安装** `langchain-community`（或等效的）包：

```bash
pip install --quiet -U langchain-community
```

2. **安装** PaymanAI SDK：
```bash
pip install paymanai
```

3. **设置**环境变量：
```bash
export PAYMAN_API_SECRET="YOUR_SECRET_KEY"
export PAYMAN_ENVIRONMENT="sandbox"
```

您的 `PAYMAN_API_SECRET` 应为 app.paymanai.com 提供的密钥。`PAYMAN_ENVIRONMENT` 可以是 `sandbox` 或 `production`，具体取决于您的使用场景。

## 实例化

以下是实例化 PaymanAI 工具的示例。如果您有多个 Payman 方法，可以创建多个工具。

```python
from langchain_community.tools.langchain_payman_tool.tool import PaymanAI

# 实例化 PaymanAI 工具（示例）
tool = PaymanAI(
    name="send_payment",
    description="将支付发送给指定的收款人。",
)
```

## 调用

### 直接带参数调用

您可以调用 `tool.invoke(...)` 并传递一个与工具预期字段匹配的字典。例如：

```python
response = tool.invoke({
    "amount_decimal": 10.00,
    "payment_destination_id": "abc123",
    "customer_id": "cust_001",
    "memo": "为发票 #XYZ 付款"
})
```

### 使用 ToolCall 调用

在 AI 工作流中使用时，LLM 可能会生成一个 `ToolCall` 字典。您可以按如下方式模拟它：

```python
model_generated_tool_call = {
    "args": {
        "amount_decimal": 10.00,
        "payment_destination_id": "abc123"
    },
    "id": "1",
    "name": tool.name,
    "type": "tool_call",
}
tool.invoke(model_generated_tool_call)
```

## 在 Chain 或 Agent 中使用工具

您可以将 PaymanAI 工具绑定到支持工具调用的 LangChain Agent 或 Chain。

## 快速入门摘要

1. 在 app.paymanai.com **注册**以获取您的 **API Key**。
2. **安装**依赖项：
   ```bash
   pip install paymanai langchain-community
   ```
3. **导出**环境变量：
   ```bash
   export PAYMAN_API_SECRET="YOUR_SECRET_KEY"
   export PAYMAN_ENVIRONMENT="sandbox"
   ```
4. **实例化**一个 PaymanAI 工具，并传入您想要的名称/描述。
5. 使用 `.invoke(...)` **调用**该工具，或将其集成到 Chain 或 Agent 中。

## API 参考文献

您可以在以下位置找到 PaymanAI 的完整 API 文档：

- [Langchain-Payman Python 参考](https://pypi.org/project/langchain-payman-tool/)
- [Payman 文档](https://docs.paymanai.com)

## 链式调用

```python
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain.chat_models import init_chat_model

# 假设我们已经导入了你的 PaymanAITool 或多个 Payman AI 工具
payman_tool = PaymanAITool(name="send_payment")

# 构建一个提示
prompt = ChatPromptTemplate([
    ("system", "你是一个乐于助人的人工智能，可以根据要求发送付款。"),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

llm = init_chat_model(model="gpt-4", model_provider="openai")
llm_with_tools = llm.bind_tools([payman_tool], tool_choice=payman_tool.name)

llm_chain = prompt | llm_with_tools

@chain
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    tool_msgs = payman_tool.batch(ai_msg.tool_calls, config=config)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 示例用法：
response = tool_chain.invoke("向 payee123 发送 10 美元。")
print(response)```

## API 参考

您可以在以下位置找到 PaymanAI 的完整 API 文档：

- [Python 参考](https://python.langchain.com/v0.2/api_reference/community/tools/langchain_community.tools.langchain_payman_tool.tool.PaymanAI.html)
- (任何其他相关的参考或文档链接)